In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, confusion_matrix
from skimage.feature import hog
from skimage.color import rgb2gray

In [2]:
# ========== Feature Extractor (Color Histogram) ==========
def extract_features(image, bins=32, resize_dim=(64, 64)):
    image = image.resize(resize_dim).convert('RGB') # Ensures that all images are RGB
    image_np = np.array(image)
    hist_r = np.histogram(image_np[:, :, 0], bins=bins, range=(0, 256))[0]
    hist_g = np.histogram(image_np[:, :, 1], bins=bins, range=(0, 256))[0]
    hist_b = np.histogram(image_np[:, :, 2], bins=bins, range=(0, 256))[0]
    hist = np.concatenate([hist_r, hist_g, hist_b])
    return hist / np.sum(hist)

In [ ]:
def extract_hog_features(image, resize_dim=(64, 64), orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2)):
    image = image.resize(resize_dim).convert('RGB')
    gray = rgb2gray(np.array(image))
    features = hog(gray, orientations=orientations,
                   pixels_per_cell=pixels_per_cell,
                   cells_per_block=cells_per_block,
                   block_norm='L2-Hys')
    return features



def extract_combined_features(image):
    hog_feat = extract_hog_features(image)
    hist_feat = extract_features(image)
    return np.concatenate([hog_feat, hist_feat])

In [ ]:
# Parameters
image_size = (64, 64)  # Resize all images to 64x64
valid_exts = ('.jpg', '.jpeg', '.png')

# Function to load images from a given folder
def load_dataset(root_dir, extractor_fn):
    X = []
    y = []
    class_names = sorted(os.listdir(root_dir))
    for label in class_names:
        label_path = os.path.join(root_dir, label)
        if not os.path.isdir(label_path):
            continue
        for fname in os.listdir(label_path):
            if fname.lower().endswith(valid_exts):
                try:
                    img_path = os.path.join(label_path, fname)
                    img = Image.open(img_path).convert('RGB')
                    if extractor_fn == None:
                        img = img.resize(image_size)
                        img_array = np.array(img).flatten()  # Flatten to 1D vector (64*64*3)
                        X.append(img_array)
                        y.append(label)
                    else:
                        features = extractor_fn(img)
                        X.append(features)
                        y.append(label)
                except Exception as e:
                    print(f"Error loading {img_path}: {e}")
    return np.array(X), np.array(y)

# Load training and testing data
print("=== Load data ===")
func_name = None #extract_features #combined features
X_train, y_train = load_dataset("Training", func_name)
X_test, y_test = load_dataset("Test", func_name)

# Encode labels (e.g., 'library' -> 0, etc.)
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

# Shuffle and standardize
X_train, y_train_enc = shuffle(X_train, y_train_enc, random_state=42)
X_test, y_test_enc = shuffle(X_test, y_test_enc, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")
print(f"Classes: {le.classes_}")

=== Load data ===
Train shape: (15000, 12288), Test shape: (300, 12288)
Classes: ['library-indoor' 'museum-indoor' 'shopping_mall-indoor']


In [ ]:
# ---- Random Forest ---- Dont run this for now
print("\n--- Random Forest Training ---")
rf_params = {'n_estimators': [50, 100, 150], 'max_depth': [5, 10, 15], 'criterion': ['gini','entropy'],'min_samples_split': [5, 10],
             'min_samples_leaf': [5, 10], 'random_state': [None,42]}
rf = GridSearchCV(RandomForestClassifier(), rf_params, scoring='accuracy')
rf.fit(X_train, y_train_enc)
rf_preds = rf.predict(X_test)
print(classification_report(y_test_enc, rf_preds))
print(confusion_matrix(y_test_enc, rf_preds))


--- Random Forest Training ---


In [ ]:
# ---- Random Forest ----
print("\n--- Random Forest Training ---")
# rf_params = {'n_estimators': [50, 100, 150], 'max_depth': [5, 10, 15], 'criterion': ['gini','entropy'],'min_samples_split': [2, 5]}
rf = RandomForestClassifier(n_estimators= 120, 
                            max_depth= 9, 
                            criterion= "gini", 
                            min_samples_split= 2,
                            min_samples_leaf=1,
                            max_features='sqrt',
                            random_state=None)
rf.fit(X_train, y_train_enc)
rf_preds = rf.predict(X_test)
print(classification_report(y_test_enc, rf_preds))
print(confusion_matrix(y_test_enc, rf_preds))


--- Random Forest Training ---
              precision    recall  f1-score   support

           0       0.58      0.52      0.55       100
           1       0.65      0.59      0.62       100
           2       0.57      0.68      0.62       100

    accuracy                           0.60       300
   macro avg       0.60      0.60      0.60       300
weighted avg       0.60      0.60      0.60       300

[[52 19 29]
 [18 59 23]
 [19 13 68]]


In [19]:
rf_preds_train = rf.predict(X_train)
print(classification_report(y_train_enc, rf_preds_train))
print(confusion_matrix(y_train_enc, rf_preds_train))

              precision    recall  f1-score   support

           0       0.84      0.87      0.86      5000
           1       0.90      0.77      0.83      5000
           2       0.80      0.89      0.84      5000

    accuracy                           0.84     15000
   macro avg       0.85      0.84      0.84     15000
weighted avg       0.85      0.84      0.84     15000

[[4338  188  474]
 [ 502 3841  657]
 [ 307  257 4436]]


In [8]:
results = rf.cv_results_
for mean, std, params in zip(results["mean_test_score"],
                              results["std_test_score"],
                              results["params"]):
    print(f"{params} --> mean: {mean:.4f}, std: {std:.4f}")

{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 50} --> mean: 0.5142, std: 0.0085
{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 100} --> mean: 0.5124, std: 0.0069
{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 200} --> mean: 0.5115, std: 0.0076
{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 50} --> mean: 0.5139, std: 0.0098
{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 100} --> mean: 0.5113, std: 0.0058
{'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 200} --> mean: 0.5149, std: 0.0078
{'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 50} --> mean: 0.5383, std: 0.0079
{'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 100} --> mean: 0.5452, std: 0.0063
{'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 200} --> mean: 0.5